<a href="https://colab.research.google.com/github/donghyundavidchoi/Data_Analysis/blob/main/%EB%AA%A8%EB%8D%B8%EB%A7%81/%EB%B3%B4%ED%97%98%EB%A3%8C%EC%98%88%EC%B8%A1%EB%AA%A8%EB%8D%B8_get_dummies_LinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --fuzzy https://drive.google.com/file/d/1txRVXp-Eh222PzIeTQ-fuAoUYFR9f7q-/view?usp=sharing
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Downloading...
From: https://drive.google.com/uc?id=1txRVXp-Eh222PzIeTQ-fuAoUYFR9f7q-
To: /content/kaggle.json
100% 73.0/73.0 [00:00<00:00, 116kB/s]


In [3]:
!kaggle datasets download -d mirichoi0218/insurance

  0% 0.00/16.0k [00:00<?, ?B/s]
100% 16.0k/16.0k [00:00<00:00, 14.7MB/s]


In [4]:
!unzip /content/insurance.zip

Archive:  /content/insurance.zip
  inflating: insurance.csv           


In [127]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("/content/insurance.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='charges')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1070, 7), (268, 7), (1070, 2), (268, 2))

--------

# EDA

In [128]:
X_train

,id,age,sex,bmi,children,smoker,region
209,209,40,male,41.230,1,no,northeast
540,540,34,female,38.000,3,no,southwest
747,747,19,male,21.755,0,no,northwest
39,39,60,male,39.900,0,yes,southwest
640,640,33,male,42.400,5,no,southwest
...,...,...,...,...,...,...,...
44,44,38,male,37.050,1,no,northeast
621,621,37,male,34.100,4,yes,southwest
1152,1152,43,female,32.560,3,yes,southeast
57,57,18,male,31.680,2,yes,southeast


In [129]:
X_train.describe()

,id,age,bmi,children
count,1070.000000,1070.000000,1070.000000,1070.000000
mean,663.761682,39.321495,30.755519,1.105607
std,385.000181,13.984804,6.138228,1.216916
min,0.000000,18.000000,15.960000,0.000000
25%,334.250000,27.000000,26.400000,0.000000
50%,656.500000,39.000000,30.500000,1.000000
75%,990.500000,51.000000,34.800000,2.000000
max,1337.000000,64.000000,53.130000,5.000000


In [130]:
X_train.isna().sum()

id          0
age         0
sex         0
bmi         0
children    0
smoker      0
region      0
dtype: int64

In [131]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1070 entries, 209 to 1140
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        1070 non-null   int64  
 1   age       1070 non-null   int64  
 2   sex       1070 non-null   object 
 3   bmi       1070 non-null   float64
 4   children  1070 non-null   int64  
 5   smoker    1070 non-null   object 
 6   region    1070 non-null   object 
dtypes: float64(1), int64(3), object(3)
memory usage: 66.9+ KB


------

# object columns 선택

In [132]:
cols = X_train.select_dtypes("object").columns

for col in cols:
    print("\n=====", col, "=====")
    print("[train]")
    print(X_train[col].value_counts())
    print("[test]")
    print(X_test[col].value_counts())


===== sex =====
[train]
male      551
female    519
Name: sex, dtype: int64
[test]
female    143
male      125
Name: sex, dtype: int64

===== smoker =====
[train]
no     845
yes    225
Name: smoker, dtype: int64
[test]
no     219
yes     49
Name: smoker, dtype: int64

===== region =====
[train]
southeast    304
northeast    266
southwest    261
northwest    239
Name: region, dtype: int64
[test]
northwest    86
southwest    64
southeast    60
northeast    58
Name: region, dtype: int64


-------

# sex와 smoker에 대해 gewt_dummies를 통해 변환

In [133]:
X_train = pd.get_dummies(X_train, columns=cols)

In [134]:
X_train

,id,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
209,209,40,41.230,1,0,1,1,0,1,0,0,0
540,540,34,38.000,3,1,0,1,0,0,0,0,1
747,747,19,21.755,0,0,1,1,0,0,1,0,0
39,39,60,39.900,0,0,1,0,1,0,0,0,1
640,640,33,42.400,5,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
44,44,38,37.050,1,0,1,1,0,1,0,0,0
621,621,37,34.100,4,0,1,0,1,0,0,0,1
1152,1152,43,32.560,3,1,0,0,1,0,0,1,0
57,57,18,31.680,2,0,1,0,1,0,0,1,0


-------

# X_test에 대해서도 X_train과 같은 전처리 진행

In [135]:
X_test = pd.get_dummies(X_test, columns=cols)

-------

# id 열 삭제

In [136]:
X_train_id = X_train['id']
X_train = X_train.drop(columns='id', axis=1)
X_test_id = X_test['id']
X_test = X_test.drop(columns='id', axis=1)

-------

# y 데이터 확인

In [137]:
y_train

,id,charges
209,209,6610.10970
540,540,6196.44800
747,747,1627.28245
39,39,48173.36100
640,640,6666.24300
...,...,...
44,44,6079.67150
621,621,40182.24600
1152,1152,40941.28540
57,57,34303.16720


-------

# 모델링
# 특별히 더 고려해야할 사항이 없으므로 LinearRegression 진행

In [140]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train['charges'])
print(model.score(X_train, y_train['charges']))
print(model.score(X_test, y_test['charges']))

0.7528339112924829
0.7387602948161295


-------

# RMSE 확인

In [141]:
from sklearn.metrics import mean_squared_error
prediction = model.predict(X_test)
mse = mean_squared_error(y_test['charges'], prediction)
rmse = mse**0.5
print("RMSE : ", rmse)

RMSE :  6069.0269218082185


-------

# 결과를 형식에 맞춰 저장

In [142]:
output = pd.DataFrame({'id' : X_test_id, 'charges' : prediction})

In [143]:
output

,id,charges
1088,1088,16873.378242
1157,1157,2517.863763
1267,1267,28597.715125
506,506,4573.440040
659,659,14429.483268
...,...,...
1275,1275,9318.495528
88,88,8875.507293
646,646,6929.470073
654,654,13592.002567


In [144]:
output.to_csv("20220609.csv", index=False)